In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.interpolate import interp1d

import sys
sys.path.append('/Users/jpw/py/spectools_ir/')
import spectools_ir
from spectools_ir.utils import extract_hitran_data, spec_convol_R
from astropy.table import Table, vstack

from astropy.io import ascii, fits
from matplotlib.gridspec import GridSpec
import pandas as pd
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

#Asys.path.append('/Users/pietro/PhD/Work/iSHELL_spectroastrometry/')
#from shift_vel_mine import CalShift

%matplotlib inline

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

source = 'HLTau'
prop = pd.read_csv(f'disk_properties.csv', skipinitialspace=True)
PA1 = prop[source][0]
PA2 = PA1+180

# From Adwin
v_geo_Ad = prop[source][1]

In [2]:
date = '240107'
path = '/Users/jpw/Analysis/NIRSPEC/iSHELL/'+date+'/'
source = 'HLTau'
PA1 = 48

source = 'CWTau'
PA1 = 61

date = '230630'
path = '/Volumes/JPW_2TB/iSHELL/'+date+'/'
source = 'Elias24'
PA1 = 136

PA2 = PA1 + 180
v_geo_Ad = -10.0

In [3]:
table1 = pd.read_csv(path + f'reduced/{source}_flux_combined_PA{PA1}_{PA2}.csv', skipinitialspace=True)
wave_flux = table1['wavelength']
wave_flux = wave_flux - wave_flux*v_geo_Ad/299792    # Correcting for motion relative to Earth
flux = table1['flux']
err_flux = table1['err_flux']

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/JPW_2TB/iSHELL/240107/reduced/Elias24_flux_combined_PA136_316.csv'

In [ ]:
fig = plt.figure(figsize=(14, 7))
ax = fig.add_subplot(111)

plt.plot(wave_flux, flux, lw=1, color='b')
plt.fill_between(wave_flux, flux-err_flux, flux+err_flux, alpha=0.5, color='b')

ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
ax.set_ylabel('Flux(Jy)', fontsize = 20, labelpad=10)
#ax.set_ylim(-0.5, 0.5)
#ax.set_xlim(4.7,4.75)

In [ ]:
def median_filter(arr, arr_err, w):
    # Median filter 1D array arr over a window size w in pixels

    n = arr.size
    arr_filter, arr_fiter_err = np.zeros(n), np.zeros(n)
    
    # Define the window
    for i in range(n):
        
        # Left edge
        if i<int((w-1)/2):
            arr_window = arr[:i+1+int((w-1)/2)] 
        # Central pixels
        if int((w-1)/2)<= i < int(n - (w-1)/2):
            arr_window = arr[i-int((w-1)/2):i+1+int((w-1)/2)]
        # Right edge
        if i>=int(n - (w-1)/2):
            arr_window = arr[i-int((w-1)/2):] 
            
        arr_filter[i] = np.nanmedian(arr_window)
        arr_fiter_err[i] = 1.253 * arr_err[i]/np.sqrt(arr_window.size)
    
    return arr_filter, arr_fiter_err

In [ ]:
##### Normalization of the spectrum #####
# We first sigma clip, then apply the median filter to the clipped data

from astropy.stats import sigma_clip
filtered_data = sigma_clip(flux, sigma=1.7, maxiters=10, masked=False, axis=0, cenfunc='median', stdfunc='mad_std')
for j in range(8):
    fig = plt.figure(figsize=(14, 3))
    ax = fig.add_subplot(111)

    plt.plot(wave_flux, filtered_data, lw=1, color='b')
    #plt.fill_between(wave_flux, flux-err_flux, flux+err_flux, alpha=0.5, color='b')

    #plt.scatter(wave_flux, flux_filter, lw=1, color='orange', zorder=2, s=1)

    ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
    ax.set_ylabel('Flux(Jy)', fontsize = 20, labelpad=10)
    #ax.set_ylim(2, 7)
    ax.set_xlim(4.5+j/10,4.6+j/10)

In [ ]:
window_width = 500 # pixels 
flux_filter, flux_filter_err = median_filter(filtered_data, err_flux, window_width)
flux_norm = flux/flux_filter
err_flux_norm = err_flux/flux_filter

In [ ]:
fig = plt.figure(figsize=(14, 3))
ax = fig.add_subplot(111)

plt.plot(wave_flux, flux, lw=1, color='b')
plt.fill_between(wave_flux, flux-err_flux, flux+err_flux, alpha=0.5, color='b')

pixel_wave_size = (wave_flux.max()- wave_flux.min())/wave_flux.size
ax.axvline(4.72, color='gray', ls='--')
ax.axvline(4.72+window_width*pixel_wave_size, color='gray', ls='--')


ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
ax.set_ylabel('Flux(Jy)', fontsize = 20, labelpad=10)
#ax.set_ylim(-0.5, 0.5)
ax.set_xlim(4.7,4.75)

In [ ]:
for j in range(8):
    fig = plt.figure(figsize=(14, 3))
    ax = fig.add_subplot(111)

    plt.plot(wave_flux, flux, lw=1, color='b')
    plt.fill_between(wave_flux, flux-err_flux, flux+err_flux, alpha=0.5, color='b')

    plt.scatter(wave_flux, flux_filter, lw=1, color='orange', zorder=2, s=1)

    ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
    ax.set_ylabel('Flux(Jy)', fontsize = 20, labelpad=10)
    ax.set_ylim(0, 4)
    ax.set_xlim(4.5+j/10,4.6+j/10)

In [ ]:
fig = plt.figure(figsize=(14, 7))
ax = fig.add_subplot(111)

plt.plot(wave_flux, flux/flux_filter, lw=1, color='b', label='Normalized spectrum')
#plt.fill_between(wave_flux, flux/flux_filter-err_flux/flux_filter, flux/flux_filter+err_flux/flux_filter, alpha=0.2, color='b')

plt.plot(wave_flux, (flux-np.mean(flux_filter))/np.mean(flux_filter)+1, lw=1, color='r', alpha=0.5, zorder=0, label='Rescaled old spectrum')

ax.axhline(1, color='gray', linestyle='dashed')
ax.legend(fontsize=15)

ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
ax.set_ylabel('Normalized flux', fontsize = 20, labelpad=10)
ax.set_ylim(0., 2)
#ax.set_xlim(4.7,4.75)

In [ ]:
for j in range(8):
    fig = plt.figure(figsize=(14, 3))
    ax = fig.add_subplot(111)

    plt.plot(wave_flux, flux_norm, lw=1, color='b')
    plt.fill_between(wave_flux, flux_norm-err_flux_norm, flux_norm+err_flux_norm, alpha=0.5, color='b')

    ax.axhline(1, color='gray', linestyle='dashed')
    ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
    ax.set_ylabel('Flux(Jy)', fontsize = 20, labelpad=10)
    ax.set_ylim(0.5, 1.5)
    ax.set_xlim(4.5+j/10,4.6+j/10)

In [ ]:
fig = plt.figure(figsize=(14, 3))
ax = fig.add_subplot(111)

plt.plot(wave_flux, flux_norm, lw=1, color='b')
plt.fill_between(wave_flux, flux_norm-err_flux_norm, flux_norm+err_flux_norm, alpha=0.2, color='b')

ax.axhline(1, color='gray', linestyle='dashed')

ax.set_xlabel(r'$\lambda$ ($\mu$m)', fontsize = 20, labelpad=10)
ax.set_ylabel('Normalized flux', fontsize = 20, labelpad=10)
ax.set_ylim(0.8, 1.3)
ax.set_xlim(4.7,4.75)

In [ ]:
# Load SA (in mas)
SA_table = pd.read_csv(path+f'rectified/{source}_SA_combined_PA{PA1}_{PA2}.csv', skipinitialspace=True)
wave_SA = SA_table['wavelength']
wave_SA = wave_SA - wave_SA*v_geo_Ad/299792    # Correcting for motion relative to Earth
SA = SA_table['SA']
err_SA = SA_table['err_SA']

In [ ]:
csvfile = open(path+f'reduced/{source}_flux_combined_PA{PA1}_{PA2}_shifted.csv', 'w')
csvfile.write('wave_flux,  flux, err_flux\n')

for i in range(wave_flux.size):
    csvfile.write(f'{wave_flux[i]:11.9f}, {flux_norm[i]:.5f}, {err_flux_norm[i]:.5f}\n')
    
csvfile.close()

In [ ]:
csvfile = open(path+f'rectified/{source}_SA_combined_PA{PA1}_{PA2}_shifted.csv', 'w')
csvfile.write('wave_SA, SA, err_SA\n')

for i in range(wave_SA.size):
    csvfile.write(f'{wave_SA[i]:11.9f}, {SA[i]:.5f}, {err_SA[i]:.5f}\n')
    
csvfile.close()

# Stacking 12CO v=1-0

In [ ]:
hitran_CO_1_0 = extract_hitran_data('CO', np.nanmin(wave_flux)-np.nanmin(wave_flux)*0.001, np.nanmax(wave_flux)+np.nanmax(wave_flux)*0.001, vup=1)
hitran_CO_2_1 = extract_hitran_data('CO', np.nanmin(wave_flux)-np.nanmin(wave_flux)*0.001, np.nanmax(wave_flux)+np.nanmax(wave_flux)*0.001, vup=2)
hitran_13CO_1_0 = extract_hitran_data('CO', np.nanmin(wave_flux)-np.nanmin(wave_flux)*0.001, np.nanmax(wave_flux)+np.nanmax(wave_flux)*0.001, isotopologue_number=2, vup=1)

hitran_CO_1_0.sort('wave')
hitran_CO_2_1.sort('wave')
hitran_13CO_1_0.sort('wave')

In [ ]:
lowJ = ['P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18'
        'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18'] 

#highJ = ['P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31']
highJ = ['P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39', 'P40', 'P41', 'P42']

# Low J

In [ ]:
# wavelength range over which to look for lines
wave_range = (np.nanmin(wave_flux)-np.nanmin(wave_flux)*0.001, np.nanmax(wave_flux)+np.nanmax(wave_flux)*0.001)

# wavelength window (in microns) around the line to look for data (roughly corresponds to +/-30 km/s)
delta_wave_signal = 5e-4

# find the transitions where there is some signal
trans_nums = 0
trans_wave = []
trans_ids = []
for i, wave1 in enumerate(hitran_CO_1_0['wave']):
    if hitran_CO_1_0['Qpp'][i].replace(' ','') in lowJ:
        if ((wave1 > wave_range[0]) & (wave1 < wave_range[1])):
            # Select indices around the transitions
            j = np.where((wave_flux > wave1-delta_wave_signal) & (wave_flux < wave1+delta_wave_signal))[0]
            # Accept and save transitions and their ids for where we have signal (not everything is NaN)
            if np.sum(np.isfinite(flux[j])) > 0:
                trans_nums += 1
                trans_wave.append(wave1)
                trans_ids.append(hitran_CO_1_0['Qpp'][i].replace(' ',''))
            
            
# take snippets of each transition from +/-delta_v in km/s, sampled at 1 km/s
# ~10% wider in wavelength to avoid interpolation issues
delta_v = 120
clight = 2.99792458e5
delta_wave = 1.1 * 5 * delta_v / clight

v_spec = np.arange(-delta_v, 1.01*delta_v, 1)
nv = v_spec.size
flux_stack_lowJ = np.zeros(nv)
weights_stack_flux_lowJ = np.zeros(nv)
    
nrows = 8
ncols = math.ceil(trans_nums / nrows)
fig, axs = plt.subplots(nrows, ncols, figsize=(18, 18))

print(f'Stacking transitions:', end=' ')
for i, wave1 in enumerate(trans_wave):
    #print(i, wave1)
    col = i % ncols
    row = int(i / ncols)

    j = np.where((wave_flux > wave1-delta_wave) & (wave_flux < wave1+delta_wave))[0]
    v_wave = (wave_flux[j] / wave1 - 1) * clight
    wave_plot = wave_flux[j]

    flux_v = interp1d(v_wave, flux_norm[j], bounds_error=False)
    flux_err = interp1d(v_wave, err_flux_norm[j], bounds_error=False)
    #axs[row, col].plot(v_spec, flux_v(v_spec), color='b')
    #axs[row, col].set_xlim(-delta_v, delta_v)

    axs[row, col].plot(wave_plot, flux_norm[j], color='darkblue')
    axs[row, col].set_xlim(wave1-delta_wave, wave1+delta_wave)
    axs[row, col].axvline(wave1, color='C1', ls = '--')
    axs[row, col].text(wave1+0.03*delta_wave, 0.6, trans_ids[i], rotation=90, fontsize=12, fontweight='bold', color='C1')

    # plot any 13CO v=1-0 transitions within delta_wave of the 13CO line
    for k, wave_CO10 in enumerate(hitran_13CO_1_0['wave']):
        if np.abs(wave_CO10 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO10, color='green', ls=':')
            axs[row, col].text(wave_CO10+0.03*delta_wave, 0.6, hitran_CO_1_0['Qpp'][i].replace(' ',''), color='green', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # plot any CO v=2-1 transitions within delta_wave of the 13CO line
    for k, wave_CO21 in enumerate(hitran_CO_2_1['wave']):
        if np.abs(wave_CO21 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO21, color='cyan', ls=':')
            axs[row, col].text(wave_CO21+0.03*delta_wave, 0.6, hitran_CO_2_1['Qpp'][i].replace(' ',''), color='cyan', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # stacking without counting the NaNs
    # do not include the following 13CO transitions since they are contaminated by some CO v=1-0 transitions
    #bad_trans_v10 = ['R38','R35','R34','R33','R30','R29','R26','R25','R24','R21','R20','R19','R15','R14','R9','R8','R7','R2','R1','R0','P6','P13','P14','P15','P22','P23','P24','P33','P34']
    #bad_trans_v21 = []
    #bad_trans = bad_trans_v10 + bad_trans_v21
    #if trans_ids[i] not in bad_trans:
    
    print(f'{trans_ids[i]},', end=' ')
    # stacking without counting the NaNs
    for j, v1 in enumerate(v_spec):
        f = flux_v(v1)
        e = flux_err(v1)
        if np.isfinite(f) & np.isfinite(e):
            w = 1/e**2
            flux_stack_lowJ[j] = flux_stack_lowJ[j] + w * f
            weights_stack_flux_lowJ[j] = weights_stack_flux_lowJ[j] + w

    axs[row, col].set_ylim(0.5, 1.5)
    #axs[row, col].axhline(1, color='k', ls=':')
    axs[row, col].set_xlabel(r'$\lambda$ ($\mu$m)')
    if col == 0:
        axs[row, col].set_ylabel('Normalized flux')

print(f'\n')

# don't show axes for panels where there is no data
n_extra = nrows * ncols - len(trans_wave)
if n_extra > 0:
    for i in range(n_extra):
        col = (nrows*ncols - i - 1) % ncols
        row = int((nrows*ncols - i - 1) / ncols)
        axs[row, col].axis('off')

#plt.savefig('transitions.png')

In [ ]:
# wavelength range over which to look for lines
wave_range = (np.nanmin(wave_SA)-np.nanmin(wave_SA)*0.001, np.nanmax(wave_SA)+np.nanmax(wave_SA)*0.001)

# wavelength window (in microns) around the line to look for data (roughly corresponds to +/-30 km/s)
delta_wave_signal = 5e-4

# find the transitions where there is some signal
trans_nums = 0
trans_wave = []
trans_ids = []
for i, wave1 in enumerate(hitran_CO_1_0['wave']):
    if hitran_CO_1_0['Qpp'][i].replace(' ','') in lowJ:
        if ((wave1 > wave_range[0]) & (wave1 < wave_range[1])):
            # Select indices around the transitions
            j = np.where((wave_SA > wave1-delta_wave_signal) & (wave_SA < wave1+delta_wave_signal))[0]
            # Accept and save transitions and their ids for where we have signal (not everything is NaN)
            if np.sum(np.isfinite(SA[j])) > 0:
                trans_nums += 1
                trans_wave.append(wave1)
                trans_ids.append(hitran_CO_1_0['Qpp'][i].replace(' ',''))
            
# take snippets of each transition from +/-delta_v in km/s, sampled at 1 km/s
# ~10% wider in wavelength to avoid interpolation issues
delta_v = 120
clight = 2.99792458e5
delta_wave = 1.1 * 5 * delta_v / clight

v_spec = np.arange(-delta_v, 1.01*delta_v, 1)
nv = v_spec.size
SA_stack_lowJ = np.zeros(nv)
weights_stack_SA_lowJ = np.zeros(nv)
    
nrows = 8
ncols = math.ceil(trans_nums / nrows)
fig, axs = plt.subplots(nrows, ncols, figsize=(18, 18))

print(f'Stacking transitions:', end=' ')
for i, wave1 in enumerate(trans_wave):
    #print(i, wave1)
    col = i % ncols
    row = int(i / ncols)

    j = np.where((wave_SA > wave1-delta_wave) & (wave_SA < wave1+delta_wave))[0]
    v_wave = (wave_SA[j] / wave1 - 1) * clight
    wave_plot = wave_SA[j]

    SA_v = interp1d(v_wave, SA[j], bounds_error=False)
    SA_err = interp1d(v_wave, err_SA[j], bounds_error=False)
    #axs[row, col].plot(v_spec, SA_v(v_spec), color='b')
    #axs[row, col].set_xlim(-delta_v, delta_v)

    axs[row, col].plot(wave_plot, SA[j], color='k')
    axs[row, col].set_xlim(wave1-delta_wave, wave1+delta_wave)
    axs[row, col].axvline(wave1, color='C1', ls = '--')
    axs[row, col].text(wave1+0.03*delta_wave, 0.6, trans_ids[i], rotation=90, fontsize=12, fontweight='bold', color='C1')

    # plot any 13CO v=1-0 transitions within delta_wave of the 13CO line
    for k, wave_CO10 in enumerate(hitran_13CO_1_0['wave']):
        if np.abs(wave_CO10 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO10, color='green', ls=':')
            axs[row, col].text(wave_CO10+0.03*delta_wave, 0.6, hitran_CO_1_0['Qpp'][i].replace(' ',''), color='green', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # plot any CO v=2-1 transitions within delta_wave of the 13CO line
    for k, wave_CO21 in enumerate(hitran_CO_2_1['wave']):
        if np.abs(wave_CO21 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO21, color='cyan', ls=':')
            axs[row, col].text(wave_CO21+0.03*delta_wave, 0.6, hitran_CO_2_1['Qpp'][i].replace(' ',''), color='cyan', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # stacking without counting the NaNs
    # do not include the following 13CO transitions since they are contaminated by some CO v=1-0 transitions
    #bad_trans_v10 = ['R38','R35','R34','R33','R30','R29','R26','R25','R24','R21','R20','R19','R15','R14','R9','R8','R7','R2','R1','R0','P6','P13','P14','P15','P22','P23','P24','P33','P34']
    #bad_trans_v21 = []
    #bad_trans = bad_trans_v10 + bad_trans_v21
    #if trans_ids[i] not in bad_trans:
    
    print(f'{trans_ids[i]},', end=' ')
    # stacking without counting the NaNs
    for j, v1 in enumerate(v_spec):
        f = SA_v(v1)
        e = SA_err(v1)
        if np.isfinite(f) & np.isfinite(e):
            w = 1/e**2
            SA_stack_lowJ[j] = SA_stack_lowJ[j] + w * f
            weights_stack_SA_lowJ[j] = weights_stack_SA_lowJ[j] + w

    axs[row, col].set_ylim(-20, 20)
    #axs[row, col].axhline(1, color='k', ls=':')
    axs[row, col].set_xlabel(r'$\lambda$ ($\mu$m)')
    if col == 0:
        axs[row, col].set_ylabel('SA (mas)')

print(f'\n')

# don't show axes for panels where there is no data
n_extra = nrows * ncols - len(trans_wave)
if n_extra > 0:
    for i in range(n_extra):
        col = (nrows*ncols - i - 1) % ncols
        row = int((nrows*ncols - i - 1) / ncols)
        axs[row, col].axis('off')

#plt.savefig('transitions.png')

In [ ]:
flux_stacked_lowJ = flux_stack_lowJ / weights_stack_flux_lowJ
err_flux_stacked_lowJ = 1/np.sqrt(weights_stack_flux_lowJ)

SA_stacked_lowJ = SA_stack_lowJ / weights_stack_SA_lowJ
err_SA_stacked_lowJ = 1/np.sqrt(weights_stack_SA_lowJ)

In [ ]:
fig = plt.figure(figsize=(10, 7))

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0., hspace=0.07)
ax1 = fig.add_subplot(211)
ax1.plot(v_spec, flux_stacked_lowJ, color='mediumblue', lw=2)
ax1.fill_between(v_spec, flux_stacked_lowJ - err_flux_stacked_lowJ, flux_stacked_lowJ + err_flux_stacked_lowJ, color='mediumblue', alpha=0.3)
#ax.set_xlabel('Velocity (km/s)')

ax2 = fig.add_subplot(212)
ax2.scatter(v_spec, SA_stacked_lowJ, color='k', s=5)
ax2.errorbar(v_spec, SA_stacked_lowJ, err_SA_stacked_lowJ, color='k', alpha=0.8, lw=0.5, capsize=2, linestyle=' ')
ax2.axhline(0, color='purple', ls = '--', lw=2, zorder=0, alpha=0.7)

ax1.set_xlim(-100, 100)
ax2.set_xlim(-100, 100)

#ax1.set_ylim(0.2, 1.3)
ax2.set_ylim(-20, 20)


### Plot design ###

index_xticks = 25
ax1.xaxis.set_major_locator(MultipleLocator(index_xticks))
ax1.xaxis.set_minor_locator(MultipleLocator(index_xticks/4))
ax2.xaxis.set_major_locator(MultipleLocator(index_xticks))
ax2.xaxis.set_minor_locator(MultipleLocator(index_xticks/4))

index_yticks1 =0.1 
ax1.yaxis.set_major_locator(MultipleLocator(index_yticks1))
ax1.yaxis.set_minor_locator(MultipleLocator(index_yticks1/4))

index_yticks2 = 5
ax2.yaxis.set_major_locator(MultipleLocator(index_yticks2))
ax2.yaxis.set_minor_locator(MultipleLocator(index_yticks2/4))

ax1.set_xticklabels([])
ax1.tick_params(which='major',axis='both',right=True,top=True, labelsize=15, pad=7,width=2.5, length=6,direction='in',color='k')
ax1.tick_params(which='minor',axis='both',right=True,top=True, labelsize=15, pad=7,width=2, length=3,direction='in',color='k')
ax2.tick_params(which='major',axis='both',right=True,top=True, labelsize=15, pad=7,width=2.5, length=6,direction='in',color='k')
ax2.tick_params(which='minor',axis='both',right=True,top=True, labelsize=15, pad=7,width=2, length=3,direction='in',color='k')

ax2.set_xlabel('Velocity (km/s)', labelpad=10, fontsize=20)
ax1.set_ylabel('Normalized flux', labelpad=10, fontsize=20)
ax2.set_ylabel('SA (mas)', labelpad=10, fontsize=20)

for side in ax1.spines.keys():  # 'top'
        ax1.spines[side].set_linewidth(2)
for side in ax2.spines.keys():  # 'top'
        ax2.spines[side].set_linewidth(2)
        
plt.suptitle(f'{source} 12CO v=1-0 low-J {date}', fontsize=20, fontweight='bold')
plt.savefig(path+f'figures/{source}_12COv=1-0_low-J_{date}.pdf', bbox_inches='tight')

In [ ]:
csvfile = open(path+f'rectified/{source}_SA_lowJ_stacked_PA{PA1}_{PA2}.csv', 'w')
csvfile.write('vel, flux_norm, err_flux_norm, SA, err_SA\n')

for i in range(v_spec.size):
    csvfile.write(f'{v_spec[i]:.3f}, {flux_stacked_lowJ[i]:.5f}, {err_flux_stacked_lowJ[i]:.5f}, {SA_stacked_lowJ[i]:.5f}, {err_SA_stacked_lowJ[i]:.5f}\n')
    
csvfile.close()

# High J

In [ ]:
# wavelength range over which to look for lines
wave_range = (np.nanmin(wave_flux)-np.nanmin(wave_flux)*0.001, np.nanmax(wave_flux)+np.nanmax(wave_flux)*0.001)

# wavelength window (in microns) around the line to look for data (roughly corresponds to +/-30 km/s)
delta_wave_signal = 5e-4

# find the transitions where there is some signal
trans_nums = 0
trans_wave = []
trans_ids = []
for i, wave1 in enumerate(hitran_CO_1_0['wave']):
    if hitran_CO_1_0['Qpp'][i].replace(' ','') in highJ:
        if ((wave1 > wave_range[0]) & (wave1 < wave_range[1])):
            # Select indices around the transitions
            j = np.where((wave_flux > wave1-delta_wave_signal) & (wave_flux < wave1+delta_wave_signal))[0]
            # Accept and save transitions and their ids for where we have signal (not everything is NaN)
            if np.sum(np.isfinite(flux[j])) > 0:
                trans_nums += 1
                trans_wave.append(wave1)
                trans_ids.append(hitran_CO_1_0['Qpp'][i].replace(' ',''))
            
# take snippets of each transition from +/-delta_v in km/s, sampled at 1 km/s
# ~10% wider in wavelength to avoid interpolation issues
delta_v = 120
clight = 2.99792458e5
delta_wave = 1.1 * 5 * delta_v / clight

v_spec = np.arange(-delta_v, 1.01*delta_v, 1)
nv = v_spec.size
flux_stack_highJ = np.zeros(nv)
weights_stack_flux_highJ = np.zeros(nv)
    
nrows = 8
ncols = math.ceil(trans_nums / nrows)
fig, axs = plt.subplots(nrows, ncols, figsize=(18, 18))

print(f'Stacking transitions:', end=' ')
for i, wave1 in enumerate(trans_wave):
    #print(i, wave1)
    col = i % ncols
    row = int(i / ncols)

    j = np.where((wave_flux > wave1-delta_wave) & (wave_flux < wave1+delta_wave))[0]
    v_wave = (wave_flux[j] / wave1 - 1) * clight
    wave_plot = wave_flux[j]

    flux_v = interp1d(v_wave, flux_norm[j], bounds_error=False)
    flux_err = interp1d(v_wave, err_flux_norm[j], bounds_error=False)
    #axs[row, col].plot(v_spec, flux_v(v_spec), color='b')
    #axs[row, col].set_xlim(-delta_v, delta_v)

    axs[row, col].plot(wave_plot, flux_norm[j], color='darkblue')
    axs[row, col].set_xlim(wave1-delta_wave, wave1+delta_wave)
    axs[row, col].axvline(wave1, color='C1', ls = '--')
    axs[row, col].text(wave1+0.03*delta_wave, 0.6, trans_ids[i], rotation=90, fontsize=12, fontweight='bold', color='C1')

    # plot any 13CO v=1-0 transitions within delta_wave of the 13CO line
    for k, wave_CO10 in enumerate(hitran_13CO_1_0['wave']):
        if np.abs(wave_CO10 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO10, color='green', ls=':')
            axs[row, col].text(wave_CO10+0.03*delta_wave, 0.6, hitran_CO_1_0['Qpp'][i].replace(' ',''), color='green', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # plot any CO v=2-1 transitions within delta_wave of the 13CO line
    for k, wave_CO21 in enumerate(hitran_CO_2_1['wave']):
        if np.abs(wave_CO21 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO21, color='cyan', ls=':')
            axs[row, col].text(wave_CO21+0.03*delta_wave, 0.6, hitran_CO_2_1['Qpp'][i].replace(' ',''), color='cyan', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # stacking without counting the NaNs
    # do not include the following 13CO transitions since they are contaminated by some CO v=1-0 transitions
    #bad_trans_v10 = ['R38','R35','R34','R33','R30','R29','R26','R25','R24','R21','R20','R19','R15','R14','R9','R8','R7','R2','R1','R0','P6','P13','P14','P15','P22','P23','P24','P33','P34']
    #bad_trans_v21 = []
    #bad_trans = bad_trans_v10 + bad_trans_v21
    #if trans_ids[i] not in bad_trans:
    
    print(f'{trans_ids[i]},', end=' ')
    # stacking without counting the NaNs
    for j, v1 in enumerate(v_spec):
        f = flux_v(v1)
        e = flux_err(v1)
        if np.isfinite(f) & np.isfinite(e):
            w = 1/e**2
            flux_stack_highJ[j] = flux_stack_highJ[j] + w * f
            weights_stack_flux_highJ[j] = weights_stack_flux_highJ[j] + w

    axs[row, col].set_ylim(0.5, 1.5)
    #axs[row, col].axhline(1, color='k', ls=':')
    axs[row, col].set_xlabel(r'$\lambda$ ($\mu$m)')
    if col == 0:
        axs[row, col].set_ylabel('Normalized flux')

print(f'\n')

# don't show axes for panels where there is no data
n_extra = nrows * ncols - len(trans_wave)
if n_extra > 0:
    for i in range(n_extra):
        col = (nrows*ncols - i - 1) % ncols
        row = int((nrows*ncols - i - 1) / ncols)
        axs[row, col].axis('off')

#plt.savefig('transitions.png')

In [ ]:
# wavelength range over which to look for lines
wave_range = (np.nanmin(wave_SA)-np.nanmin(wave_SA)*0.001, np.nanmax(wave_SA)+np.nanmax(wave_SA)*0.001)

# wavelength window (in microns) around the line to look for data (roughly corresponds to +/-30 km/s)
delta_wave_signal = 5e-4

# find the transitions where there is some signal
trans_nums = 0
trans_wave = []
trans_ids = []
for i, wave1 in enumerate(hitran_CO_1_0['wave']):
    if hitran_CO_1_0['Qpp'][i].replace(' ','') in highJ:
        if ((wave1 > wave_range[0]) & (wave1 < wave_range[1])):
            # Select indices around the transitions
            j = np.where((wave_SA > wave1-delta_wave_signal) & (wave_SA < wave1+delta_wave_signal))[0]
            # Accept and save transitions and their ids for where we have signal (not everything is NaN)
            if np.sum(np.isfinite(SA[j])) > 0:
                trans_nums += 1
                trans_wave.append(wave1)
                trans_ids.append(hitran_CO_1_0['Qpp'][i].replace(' ',''))
            
# take snippets of each transition from +/-delta_v in km/s, sampled at 1 km/s
# ~10% wider in wavelength to avoid interpolation issues
delta_v = 120
clight = 2.99792458e5
delta_wave = 1.1 * 5 * delta_v / clight

v_spec = np.arange(-delta_v, 1.01*delta_v, 1)
nv = v_spec.size
SA_stack_highJ = np.zeros(nv)
weights_stack_SA_highJ = np.zeros(nv)
    
nrows = 2
ncols = math.ceil(trans_nums / nrows)
fig, axs = plt.subplots(nrows, ncols, figsize=(18, 10))

print(f'Stacking transitions:', end=' ')
for i, wave1 in enumerate(trans_wave):
    #print(i, wave1)
    col = i % ncols
    row = int(i / ncols)

    j = np.where((wave_SA > wave1-delta_wave) & (wave_SA < wave1+delta_wave))[0]
    v_wave = (wave_SA[j] / wave1 - 1) * clight
    wave_plot = wave_SA[j]

    SA_v = interp1d(v_wave, SA[j], bounds_error=False)
    SA_err = interp1d(v_wave, err_SA[j], bounds_error=False)
    #axs[row, col].plot(v_spec, SA_v(v_spec), color='b')
    #axs[row, col].set_xlim(-delta_v, delta_v)

    axs[row, col].plot(wave_plot, SA[j], color='k')
    axs[row, col].set_xlim(wave1-delta_wave, wave1+delta_wave)
    axs[row, col].axvline(wave1, color='C1', ls = '--')
    axs[row, col].text(wave1+0.03*delta_wave, 0.6, trans_ids[i], rotation=90, fontsize=12, fontweight='bold', color='C1')

    # plot any 13CO v=1-0 transitions within delta_wave of the 13CO line
    for k, wave_CO10 in enumerate(hitran_13CO_1_0['wave']):
        if np.abs(wave_CO10 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO10, color='green', ls=':')
            axs[row, col].text(wave_CO10+0.03*delta_wave, 0.6, hitran_CO_1_0['Qpp'][i].replace(' ',''), color='green', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # plot any CO v=2-1 transitions within delta_wave of the 13CO line
    for k, wave_CO21 in enumerate(hitran_CO_2_1['wave']):
        if np.abs(wave_CO21 - wave1) < delta_wave:
            axs[row, col].axvline(wave_CO21, color='cyan', ls=':')
            axs[row, col].text(wave_CO21+0.03*delta_wave, 0.6, hitran_CO_2_1['Qpp'][i].replace(' ',''), color='cyan', rotation=90)
    #axs[row, col].set_ylim(0.2, 1.4)

    # stacking without counting the NaNs
    # do not include the following 13CO transitions since they are contaminated by some CO v=1-0 transitions
    #bad_trans_v10 = ['R38','R35','R34','R33','R30','R29','R26','R25','R24','R21','R20','R19','R15','R14','R9','R8','R7','R2','R1','R0','P6','P13','P14','P15','P22','P23','P24','P33','P34']
    #bad_trans_v21 = []
    #bad_trans = bad_trans_v10 + bad_trans_v21
    #if trans_ids[i] not in bad_trans:
    
    print(f'{trans_ids[i]},', end=' ')
    # stacking without counting the NaNs
    for j, v1 in enumerate(v_spec):
        f = SA_v(v1)
        e = SA_err(v1)
        if np.isfinite(f) & np.isfinite(e):
            w = 1/e**2
            SA_stack_highJ[j] = SA_stack_highJ[j] + w * f
            weights_stack_SA_highJ[j] = weights_stack_SA_highJ[j] + w

    axs[row, col].set_ylim(-20, 20)
    #axs[row, col].axhline(1, color='k', ls=':')
    axs[row, col].set_xlabel(r'$\lambda$ ($\mu$m)')
    if col == 0:
        axs[row, col].set_ylabel('SA (mas)')

print(f'\n')

# don't show axes for panels where there is no data
n_extra = nrows * ncols - len(trans_wave)
if n_extra > 0:
    for i in range(n_extra):
        col = (nrows*ncols - i - 1) % ncols
        row = int((nrows*ncols - i - 1) / ncols)
        axs[row, col].axis('off')

#plt.savefig('transitions.png')

In [ ]:
flux_stacked_highJ = flux_stack_highJ / weights_stack_flux_highJ
err_flux_stacked_highJ = 1/np.sqrt(weights_stack_flux_highJ)

SA_stacked_highJ = SA_stack_highJ / weights_stack_SA_highJ
err_SA_stacked_highJ = 1/np.sqrt(weights_stack_SA_highJ)

In [ ]:
fig = plt.figure(figsize=(10, 7))

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0., hspace=0.07)
ax1 = fig.add_subplot(211)
ax1.plot(v_spec, flux_stacked_highJ, color='mediumblue', lw=2)
ax1.fill_between(v_spec, flux_stacked_highJ - err_flux_stacked_highJ, flux_stacked_highJ + err_flux_stacked_highJ, color='mediumblue', alpha=0.3)
#ax.set_xlabel('Velocity (km/s)')

ax2 = fig.add_subplot(212)
ax2.scatter(v_spec, SA_stacked_highJ, color='k', s=5)
ax2.errorbar(v_spec, SA_stacked_highJ, err_SA_stacked_highJ, color='k', alpha=0.8, lw=0.5, capsize=2, linestyle=' ')
ax2.axhline(0, color='purple', ls = '--', lw=2, zorder=0, alpha=0.7)

ax1.set_xlim(-100, 100)
ax2.set_xlim(-100, 100)

#ax1.set_ylim(0.2, 1.3)
ax2.set_ylim(-10, 10)


### Plot design ###

index_xticks = 25
ax1.xaxis.set_major_locator(MultipleLocator(index_xticks))
ax1.xaxis.set_minor_locator(MultipleLocator(index_xticks/4))
ax2.xaxis.set_major_locator(MultipleLocator(index_xticks))
ax2.xaxis.set_minor_locator(MultipleLocator(index_xticks/4))

index_yticks1 = 0.05 
ax1.yaxis.set_major_locator(MultipleLocator(index_yticks1))
ax1.yaxis.set_minor_locator(MultipleLocator(index_yticks1/4))

index_yticks2 = 5
ax2.yaxis.set_major_locator(MultipleLocator(index_yticks2))
ax2.yaxis.set_minor_locator(MultipleLocator(index_yticks2/4))

ax1.set_xticklabels([])
ax1.tick_params(which='major',axis='both',right=True,top=True, labelsize=15, pad=7,width=2.5, length=6,direction='in',color='k')
ax1.tick_params(which='minor',axis='both',right=True,top=True, labelsize=15, pad=7,width=2, length=3,direction='in',color='k')
ax2.tick_params(which='major',axis='both',right=True,top=True, labelsize=15, pad=7,width=2.5, length=6,direction='in',color='k')
ax2.tick_params(which='minor',axis='both',right=True,top=True, labelsize=15, pad=7,width=2, length=3,direction='in',color='k')

ax2.set_xlabel('Velocity (km/s)', labelpad=10, fontsize=20)
ax1.set_ylabel('Normalized flux', labelpad=10, fontsize=20)
ax2.set_ylabel('SA (mas)', labelpad=10, fontsize=20)

for side in ax1.spines.keys():  # 'top'
        ax1.spines[side].set_linewidth(2)
for side in ax2.spines.keys():  # 'top'
        ax2.spines[side].set_linewidth(2)

plt.suptitle(f'{source} 12CO v=1-0 high-J {date}', fontsize=20, fontweight='bold')
plt.savefig(path+f'figures/{source}_12COv=1-0_high-J_{date}.pdf', bbox_inches='tight')

In [ ]:
csvfile = open(path+f'rectified/{source}_SA_highJ_stacked_PA{PA1}_{PA2}.csv', 'w')
csvfile.write('vel, flux_norm, err_flux_norm, SA, err_SA\n')

for i in range(v_spec.size):
    csvfile.write(f'{v_spec[i]:.3f}, {flux_stacked_highJ[i]:.5f}, {err_flux_stacked_highJ[i]:.5f}, {SA_stacked_highJ[i]:.5f}, {err_SA_stacked_highJ[i]:.5f}\n')
    
csvfile.close()